In [1]:
from functionUtils import *
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook, tnrange
import pandas as pd
import numpy as np
import warnings
import datetime
import gc
DATA_PATH = './datasets/'
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
df_train = pd.read_csv(DATA_PATH+'train.csv')
df_test = pd.read_csv(DATA_PATH+'test.csv')
df_historical = pd.read_csv(DATA_PATH+'historical_transactions.csv',dtype={'purchase_date':np.str},low_memory=True)
df_new_merchant = pd.read_csv(DATA_PATH+'new_merchant_transactions.csv',dtype={'purchase_date':np.str},low_memory=True)

df_train['is_test'] = 0
df_test['is_test'] = 1
#用户在历史和之后的记录中是否出现过
for df in [df_train,df_test]:
    df['is_in_new_monthlag1'] = df['card_id'].isin(df_new_merchant[df_new_merchant.month_lag==1].card_id).astype(int)
    df['is_in_new_monthlag2'] = df['card_id'].isin(df_new_merchant[df_new_merchant.month_lag==2].card_id).astype(int)
    df['is_in_historical'] = df['card_id'].isin(df_historical.card_id).astype(int)
df_data = pd.concat([df_train,df_test])
df_transactions = pd.concat([df_historical,df_new_merchant])

In [3]:
## id类特征
df_temp = df_historical.card_id.value_counts().reset_index()
df_temp.rename(columns={'index':'card_id','card_id':'hist_card_id_counts'},inplace=True)
df_temp['hist_card_id_ratio'] = df_temp['hist_card_id_counts']/df_historical.shape[0]
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_temp = df_new_merchant.card_id.value_counts().reset_index()
df_temp.rename(columns={'index':'card_id','card_id':'new_card_id_counts'},inplace=True)
df_temp['new_card_id_ratio'] = df_temp['new_card_id_counts']/df_new_merchant.shape[0]
df_data = df_data.merge(df_temp,on='card_id',how='left')
df_data['hist_new_card_id_ratio'] = df_data['hist_card_id_counts']/df_data['new_card_id_counts']
df_data[['hist_card_id_counts','hist_card_id_ratio','new_card_id_counts','new_card_id_ratio','hist_new_card_id_ratio']].fillna(0,inplace=True)

df_temp = df_historical.groupby('card_id')['purchase_amount'].count().reset_index()
df_temp.rename(columns={'purchase_amount':'hist_purchase_records'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_temp = df_new_merchant.groupby('card_id')['purchase_amount'].count().reset_index()
df_temp.rename(columns={'purchase_amount':'new_purchase_records'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_data['hist_new_purchase_radtio'] = df_data['hist_purchase_records']/df_data['new_purchase_records']

del df_temp
gc.collect()

98

#### 1.1 user features:时间特征&统计特征

In [4]:
def dateUtils(df=None,timeCol='purchase_date'):
    dateHandle = pd.to_datetime(df[timeCol])
    df['week'] = dateHandle.dt.week
    df['year'] = dateHandle.dt.year
    df['month_gap'] = (dateHandle.dt.date - datetime.date(2018,2,28)).dt.days//30
    df['day_gap'] = (dateHandle.dt.date - datetime.date(2018,2,28)).dt.days
    #cardid用户连续购买之间的时间差
    roll = df.groupby(['card_id'])['day_gap'].apply(lambda series:series.diff(1))
    df['day_diff'] = roll.values
    return df

dateHandle = pd.to_datetime(df_data['first_active_month'])
df_data['active_year'] = dateHandle.dt.year
df_data['active_month'] = dateHandle.dt.month
df_data['active_to_base_time'] = (datetime.date(2018,4,30) - dateHandle.dt.date).dt.days

#第一次购买距离注册的时间
df_temp = df_historical.groupby('card_id')['purchase_date'].min().reset_index()
df_temp.rename(columns={'purchase_date':'historical_first_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')
#最后一次购买时间
df_temp = df_historical.groupby('card_id')['purchase_date'].max().reset_index()
df_temp.rename(columns={'purchase_date':'historical_last_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_temp = df_new_merchant.groupby('card_id')['purchase_date'].min().reset_index()
df_temp.rename(columns={'purchase_date':'new_first_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')
#最后一次购买时间
df_temp = df_new_merchant.groupby('card_id')['purchase_date'].max().reset_index()
df_temp.rename(columns={'purchase_date':'new_last_purchase_date'},inplace=True)
df_data = df_data.merge(df_temp,on='card_id',how='left')

df_data['historical_first_to_base_time'] = (pd.to_datetime(df_data['historical_first_purchase_date']).dt.date -datetime.date(2018,4,30)).dt.days
df_data['historical_last_to_base_time'] = (pd.to_datetime(df_data['historical_last_purchase_date']).dt.date -datetime.date(2018,4,30)).dt.days
df_data['historical_last_to_first_time'] = (pd.to_datetime(df_data['historical_last_purchase_date']).dt.date - pd.to_datetime(df_data['historical_first_purchase_date']).dt.date).dt.days
df_data['historical_first_active_time'] = (pd.to_datetime(df_data['historical_first_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['historical_last_active_time'] = (pd.to_datetime(df_data['historical_last_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days

df_data['new_first_to_base_time'] = (pd.to_datetime(df_data['new_first_purchase_date']).dt.date - datetime.date(2018,4,30)).dt.days
df_data['new_last_to_base_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - datetime.date(2018,4,30)).dt.days
df_data['new_first_to_active_time'] = (pd.to_datetime(df_data['new_first_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['new_last_to_active_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - pd.to_datetime(df_data['first_active_month']).dt.date).dt.days
df_data['new_last_to_first_time'] = (pd.to_datetime(df_data['new_last_purchase_date']).dt.date - pd.to_datetime(df_data['new_first_purchase_date']).dt.date).dt.days

df_data['hist_per_time_purchaseCounts'] = df_data['hist_card_id_counts']/df_data['historical_last_to_first_time']
df_data['new_per_time_purchaseCounts'] = df_data['new_card_id_counts']/df_data['new_last_to_first_time']


df_data.drop(columns=['historical_last_purchase_date','historical_first_purchase_date','new_first_purchase_date','new_last_purchase_date'],inplace=True)

del df_train,df_test,df_historical,df_new_merchant,df_temp
gc.collect()

263

In [5]:
#缺失值处理
df_transactions['category_2'].fillna(1,inplace=True)
df_transactions['category_3'].fillna('B',inplace=True)
df_transactions['merchant_id'].fillna(df_transactions['merchant_id'].value_counts().index[0],inplace=True)
df_transactions['installments'].replace(999,np.nan,inplace=True)
df_transactions['installments'].fillna(df_transactions['installments'].value_counts().index[0],inplace=True)
df_transactions['installments'].replace(-1,np.nan,inplace=True)
df_transactions['installments'].fillna(1,inplace=True)
#交互特征
df_transactions['authorized_C1'] = df_transactions['authorized_flag'] + '_' + df_transactions['category_1']
df_transactions['C1_3'] = df_transactions['category_1'].astype(np.str) + '_' + df_transactions['category_3'].astype(np.str)
df_transactions['state_C2'] = df_transactions['state_id'].astype(np.str) + '_'  + df_transactions['category_2'].astype(np.str)
df_transactions['state_city'] = df_transactions['city_id'] + df_transactions['state_id']

for cate in ['A','B','C']:
    df_transactions['category_3_%s'%cate] = (df_transactions['category_3']==cate) + 0
for cate in [1,2,3,4,5]:
    df_transactions['category_2_%s'%cate] = (df_transactions['category_2']==cate) + 0
for cate in ['Y','N']:
    df_transactions['authorized_flag_%s'%cate] = (df_transactions['authorized_flag']==cate) + 0

df_transactions['category_1'] = df_transactions['category_1'].map({'Y':1,'N':0})
df_transactions['category_3'] = df_transactions['category_3'].map({'A':0,'B':1,'C':2})
df_transactions['authorized_flag'] = df_transactions['authorized_flag'].map({'Y':1,'N':0})

In [6]:
%%time
df_transactions.sort_values(by=['card_id','merchant_id','purchase_date'],ascending=True,inplace=True)
df_transactions = dateUtils(df_transactions,timeCol='purchase_date')
df_transactions = label_encoding(df_transactions,['authorized_C1','C1_3','state_subsector_C2','C2_cityId'])
df_transactions = downCast_dtype(df_transactions)


CPU times: user 12min 59s, sys: 1min 54s, total: 14min 53s
Wall time: 11min 50s


In [7]:
df_historical_transactions = df_transactions[df_transactions.month_lag<=0]
df_historical_auth_N = df_historical_transactions[df_historical_transactions.authorized_flag==0]
df_historical_auth_Y = df_historical_transactions[df_historical_transactions.authorized_flag==1]
df_new_transactions = df_transactions[df_transactions.month_lag>0]

In [8]:
#用户月消费记录
def getMonthPurchase(df_data,df_feature,group='month_gap',fea='purchase_amount',name='hist'):
    df_purchase = df_feature.groupby(['card_id','month_gap'])['purchase_amount'].sum().reset_index()
    df_purchase.rename(columns={'purchase_amount':'purchase_amount_sum_month'},inplace=True)
    df_purchase.sort_values(by=['month_gap'],inplace=True,ascending=True)
    df_count = df_purchase.groupby(['card_id'])['purchase_amount_sum_month'].count().reset_index()
    df_count.rename(columns={'purchase_amount_sum_month':'%s_purchase_counts'%name},inplace=True)

    df_temp = df_purchase[['card_id','month_gap','purchase_amount_sum_month']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['month_gap'],append=True)
    df_temp = pd.Series(df_temp['purchase_amount_sum_month'].values.reshape(len(df_temp['purchase_amount_sum_month'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['month_gap'].unique()):
        cols.append('%s_month%s_purchase'%(name,index))
    df_temp.columns = cols
    df_temp = df_temp.merge(df_count,on='card_id',how='left')
    df_temp.fillna(0,inplace=True)
    df_temp['%s_purchase_mean'%name] = 0
    for index in list(df_purchase['month_gap'].unique()):
        df_temp['%s_purchase_mean'%name] += df_temp['%s_month%s_purchase'%(name,index)]
    if name=='new':
        df_temp['%s_purchase_mean'%name] = (df_temp['%s_purchase_mean'%name] - df_temp['%s_month1_purchase'%name])/df_temp['%s_purchase_counts'%name]
        df_temp['base_diff_%s_purchase'%name] = df_temp['%s_month1_purchase'%name] - df_temp['%s_purchase_mean'%name]
    else:
        df_temp['%s_purchase_mean'%name] = (df_temp['%s_purchase_mean'%name] - df_temp['%s_month0_purchase'%name])/df_temp['%s_purchase_counts'%name]
        df_temp['base_diff_%s_purchase'%name] = df_temp['%s_month0_purchase'%name] - df_temp['%s_purchase_mean'%name]
    
    df_temp['base_diff_%s_purchaseRatio'%name] = df_temp['base_diff_%s_purchase'%name]/df_temp['%s_purchase_mean'%name]
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    
    del df_purchase,df_temp,cols,df_count
    gc.collect()
    
    return df_data

#周消费记录
def getWeekPurchase(df_data,df_feature,group='week',fea='purchase_amount',name='hist'):
    df_purchase = df_feature.groupby(['card_id','week'])['purchase_amount'].sum().reset_index()
    df_purchase.rename(columns={'purchase_amount':'purchase_sum_week'},inplace=True)
    df_purchase.sort_values(by=['week'],inplace=True,ascending=True)
    df_count = df_purchase.groupby(['card_id'])['purchase_sum_week'].count().reset_index()
    df_count.rename(columns={'purchase_sum_week':'%s_purchase_counts'%name},inplace=True)

    df_temp = df_purchase[['card_id','week','purchase_sum_week']]
    df_temp.index = df_temp.card_id
    df_temp = df_temp.set_index(['week'],append=True)
    df_temp = pd.Series(df_temp['purchase_sum_week'].values.reshape(len(df_temp['purchase_sum_week'])),index=df_temp.index)
    df_temp = df_temp.unstack()
    df_temp.reset_index(inplace=True)
    cols = ['card_id']
    for index in list(df_purchase['week'].unique()):
        cols.append('%s_week%s_purchase'%(name,index))
    df_temp.columns = cols
    df_temp = df_temp.merge(df_count,on='card_id',how='left')
    df_temp.fillna(0,inplace=True)
    df_data = df_data.merge(df_temp,on='card_id',how='left')
    return df_data

df_data = getMonthPurchase(df_data,df_historical_auth_Y,name='AuthY')
df_data = getMonthPurchase(df_data,df_historical_auth_N,name='AuthN')
df_data = getMonthPurchase(df_data,df_new_transactions,name='new')

df_data = getWeekPurchase(df_data,df_historical_auth_Y,name='AuthY')
df_data = getWeekPurchase(df_data,df_historical_auth_N,name='AuthN')
df_data = getWeekPurchase(df_data,df_new_transactions,name='new')

df_data.replace([np.inf,-np.inf],np.nan,inplace=True)
df_data.fillna(0,inplace=True)

In [ ]:
%%time

for flag,df_features in zip(['histAuth_Y','histAuth_N','new'],[df_historical_auth_Y,df_historical_auth_N,df_new_transactions]):
    
    print(".................%s......................"%flag)
    if 'hist'in flag:
        month_lag = [-12,-6,-3,0]
    else:
        month_lag = [1,2]
    for index in month_lag:
        df = df_features[df_features.month_lag>=index]
        #购买量
        df_data = getMeanStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMean_%s'%(flag,index))
        df_data = getMaxStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMax_%s'%(flag,index))
        df_data = getMedianStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMedian_%s'%(flag,index))
        df_data = getMinStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountMin_%s'%(flag,index))
        df_data = getSumStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purchaseAmountSum_%s'%(flag,index))
        df_data = getStdStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purcahseAmountStd_%s'%(flag,index))
        df_data = getCountsStaticsFeatures(df_data,df,['card_id'],'purchase_amount',name='%s_purcahseAmountCount_%s'%(flag,index))

        #分期数
        df_data = getMeanStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsMean_%s'%(flag,index))
        df_data = getMaxStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsMax_%s'%(flag,index))
        df_data = getSumStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsSum_%s'%(flag,index))
        df_data = getMinStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsMin_%s'%(flag,index))
        df_data = getStdStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsStd_%s'%(flag,index))
        df_data = getCountsStaticsFeatures(df_data,df,['card_id'],'installments',name='%s_installmentsCount_%s'%(flag,index))

#     #month_lag
#     df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'month_lag',name='%s_monthLagMin'%flag)
#     df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'month_lag',name='%s_monthLagMax'%flag)
#     df_data = getCategoryCounts(df_data,df_features,['card_id'],'month_lag',name='%s_monthLag_categoryCounts'%flag)
#     df_data = getCategoryCountsRatio(df_data,df_features,['card_id'],'month_lag',name='%s_monthLag_categoryCountsRatio'%flag)
    
    #day_diff
    df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffMax'%flag)
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffMin'%flag)
    df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiffFrequenceMax'%flag)
    df_data = getCategoryCounts(df_data,df_features,['card_id'],'day_diff',name='%s_dayDiff_categoryCounts'%flag)

    #day_gap
    df_data = getMaxStaticsFeatures(df_data,df_features,['card_id'],'day_gap',name='%s_purchaseToBaseTimeMax'%flag)
    df_data = getMinStaticsFeatures(df_data,df_features,['card_id'],'day_gap',name='%s_purchaseToBaseTimeMin'%flag)
    df_data = getMeanStaticsFeatures(df_data,df_features,['card_id'],'day_gap',name='%s_purchaseToBaseTimeMean'%flag)
    #类别特征
    for cate in ['category_3_A','category_3_B','category_3_C']:
        df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],cate,name='%s_%s_sum'%(flag,cate))
    for cate in [1,2,3,4,5]:
        df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'category_2_%s'%cate,name='%s_category_2_%s_sum'%(flag,cate))
    for cate in ['Y','N']:
        df_data = getSumStaticsFeatures(df_data,df_features,['card_id'],'authorized_flag_%s'%cate,name='%s_authorized_flag_%s_sum'%(flag,cate))
    
    categoryCols = ['authorized_flag','merchant_id','city_id','category_1','category_2','category_3','merchant_category_id','subsector_id',
                    'state_subsector_C2','C1_3','C2_cityId','authorized_C1']

    for fea in categoryCols:
        df_data = getCategoryFrequenceMax(df_data,df_features,['card_id'],fea,name='%s_%s_frequenceMax'%(flag,fea))
        df_data = getCategoryCounts(df_data,df_features,['card_id'],fea,name='%s_%s_categoryCounts'%(flag,fea))
        df_data = getCategoryFrequenceMaxRatio(df_data,df_features,['card_id'],fea,name='%s_%s_frequenceMaxRatio'%(flag,fea))
        df_data = getCategoryCountsRatio(df_data,df_features,['card_id'],fea,name='%s_%s_categoryCountsRatio'%(flag,fea))

.................histAuth_Y......................


In [ ]:
#保存用户统计特征
df_data.to_csv('./datasets/df_data.csv',index=False)
df_transactions.to_csv('./datasets/df_transactions.csv',index=False)

In [ ]:
df_train = df_data[df_data.is_test==0]
df_test = df_data[df_data.is_test==1]

df_train.to_csv('./datasets/df_train.csv',index=False)
df_test.to_csv('./datasets/df_test.csv',index=False)